# Modify Forcing data

**Date:** 2026-01-21

**Authors:** Maximilian Manaila

**Summary:** This notebook provides tools to load, modify and save forcing input (Simstrat-like `Forcing.dat`)

## Before you start:

**No Python knowledge needed!** Just follow these steps:
1. Run each cell in order (click the ▶️ button or press Shift+Enter)
2. Look for the sections marked "👉 CUSTOMIZE" to modify parameters

## Content:

This notebook helps you modify the forcing input. You can:

1. **Load and display forcing data** Using data downloaded from https://www.alplakes.eawag.ch/downloads
2. **Modify forcing data** Standard mathematical operations on data
3. **Save modified forcing data** To new or existing file

## Available functions:

- `data, units = load_forcing_data(path, file, start='file_start', stop='file_stop', load_units=True)` — Load downloaded forcing data and optionally load units of forcing variables
- `save_forcing_data(data, local_path, file_name, copy=True)` — Save modified forcing data to a new or a existing file
- `data = check_negative(data, val_save, var)` — Check for negative values for a forcing variable

---

# Step 1: Import Required Libraries
## ESSENTIAL STEP

Here you import the python libraries and custom functions needed for the following steps.

Simply run this cell once. You don't need to change anything here.

In [1]:
# Import standard Python libraries for data handling
import os               # Used to work with files and folders
import numpy as np      # Used for mathematical operations and working with arrays of numbers
import pandas as pd     # Used to read and modify data

# Import custom functions specifically designed for Alplakes data
# These functions do the heavy lifting for fetching and saving data
from Modify_input_functions import load_forcing_data, save_forcing_data, check_negative

print("✓ All libraries loaded successfully!")
print("You're ready to start modifying forcing data.")

✓ All libraries loaded successfully!
You're ready to start modifying forcing data.


# Step 2
## Load local data

Here you load the forcing data from your locally saved file and display the units of the forcing data as well as the data to edit.

**👉 CUSTOMIZE (next cell)** You can change these parameters:
- `local_path`: the path to the forcing data file
- `file_name`: the name of the forcing data file
    - usually `Forcing.dat`
- `start`: from when on to edit forcing data
  - date and time in format `'YYYYMMDDHHMM'` (Year-Month-Day-Hour-Minute)
  - example: `'202510231500'` = October 23, 2025 at 15:00 (3:00 PM)
  - default: `'file_start'` -> start at beginning of file
- `stop`: until when to edit forcing data
  - same format as `start`
  - default: `'file_stop'` -> stop at end of file
- `load_units`: whether to load the units from the file
    - `True` to load the units (default)
    - `False` to not load the units, returns an empty DataFrame to `units`

In [2]:
local_path = '../tests/TestCase_LakeZurich'                 # 👉 CUSTOMIZE path to the local file
file_name = 'MeteoForcing.dat'                              # 👉 CUSTOMIZE name of the local file

data, units = load_forcing_data(
    path = local_path,                                      # input path to the local file
    file = file_name,                                       # input name of the local file
    start = '202306010000',                                 # 👉 CUSTOMIZE start of model output YYYYMMDDHHMM, or file_start
    stop = 'file_stop',                                     # 👉 CUSTOMIZE end of model output YYYYMMDDHHMM, or file_stop
    load_units = True                                       # 👉 CUSTOMIZE whether to load units or not
)

display(units)                                              # display the units
display(data.round(4))                                      # display the data filtered to the date range

pos_vars = ['sol', 'vap', 'cloud', 'rain']                  # define variables that cannot be negative

Loading data from local file: ../geneva\Forcing.dat
✓ Loaded 721 rows from ../geneva\Forcing.dat
✓ Filtered to date range: 721 rows between 2023-06-01 and 2023-07-01


,u,v,Tair,sol,vap,cloud,rain
Unit,m/s,m/s,°C,W/m2,mbar,-,m/hr


,Datetime,u,v,Tair,sol,vap,cloud,rain
0,2023-06-01 00:00:00,-0.8205,-1.9331,16.546,1.0,12.6,0.1731,0.0
1,2023-06-01 01:00:02,-0.9271,-1.5429,15.746,1.0,12.7,0.1427,0.0
2,2023-06-01 01:59:57,-0.7162,-2.0801,15.546,1.0,13.0,0.1218,0.0
3,2023-06-01 03:00:00,-0.8450,-1.5893,14.946,1.0,13.0,0.1135,0.0
4,2023-06-01 04:00:02,-0.7118,-1.7616,14.346,3.0,13.1,0.1005,0.0
...,...,...,...,...,...,...,...,...
716,2023-06-30 19:59:57,-1.1476,0.3508,18.046,2.0,15.4,0.6593,0.0
717,2023-06-30 21:00:00,-0.7128,-0.3632,17.746,1.0,14.9,0.6577,0.0
718,2023-06-30 22:00:02,-1.3388,0.4093,17.746,1.0,14.3,0.6590,0.0
719,2023-06-30 22:59:57,-0.7690,0.2205,17.146,1.0,15.3,0.7016,0.0


# Step 3
## Modify forcing data

This step presents various way to modify the forcing data.

You can modify the data in the following ways:
- `Add a constant`
- `Multiply by a constant`
- `Set to a constant`
- `Add a linear trend`

Be aware that:
- The variables `'sol'`, `'vap'`, `'cloud'` and `'rain'` cannot be negative
    - this is controlled in this code

### Add constant

Here you add a constant to the any chosen forcing input.

**👉 CUSTOMIZE (next cell)** You can change these parameters:
- `variable`: choose the variable to modify from:
    - `'u'`, `'v'`, `'Tair'`, `'sol'`, `'vap'`, `'cloud'`, `'rain'`
- `constant`: the constant to add
    - any real number (can also be negative and/or a fraction)
    - variables `'sol'`, `'vap'`, `'cloud'` and `'rain'` cannot become negative

In [3]:
variable = 'u'                                      # 👉 CUSTOMIZE the variable
constant = -3                                       # 👉 CUSTOMIZE the constant

val_save = data[variable].copy()                    # save the current values
data[variable] += float(constant)                   # modify the data
if variable in pos_vars:
    data[variable] = check_negative(                # check for negative values
    data[variable], val_save, variable)
else:
    print(f"✓ Forcing variable '{variable}' modified")
display(data.round(4))                              # display the data

✓ Forcing variable 'u' modified


,Datetime,u,v,Tair,sol,vap,cloud,rain
0,2023-06-01 00:00:00,-3.8205,-1.9331,16.546,1.0,12.6,0.1731,0.0
1,2023-06-01 01:00:02,-3.9271,-1.5429,15.746,1.0,12.7,0.1427,0.0
2,2023-06-01 01:59:57,-3.7162,-2.0801,15.546,1.0,13.0,0.1218,0.0
3,2023-06-01 03:00:00,-3.8450,-1.5893,14.946,1.0,13.0,0.1135,0.0
4,2023-06-01 04:00:02,-3.7118,-1.7616,14.346,3.0,13.1,0.1005,0.0
...,...,...,...,...,...,...,...,...
716,2023-06-30 19:59:57,-4.1476,0.3508,18.046,2.0,15.4,0.6593,0.0
717,2023-06-30 21:00:00,-3.7128,-0.3632,17.746,1.0,14.9,0.6577,0.0
718,2023-06-30 22:00:02,-4.3388,0.4093,17.746,1.0,14.3,0.6590,0.0
719,2023-06-30 22:59:57,-3.7690,0.2205,17.146,1.0,15.3,0.7016,0.0


### Multiply by constant

Here you multiply any chosen forcing input by a constant.

**👉 CUSTOMIZE (next cell)** You can change these parameters:
- `variable`: choose the variable to modify from:
    - `'u'`, `'v'`, `'Tair'`, `'sol'`, `'vap'`, `'cloud'`, `'rain'`
- `constant`: the constant to multiply by
    - any real number (can also be negative and/or a fraction)
    - variables `'sol'`, `'vap'`, `'cloud'` and `'rain'` cannot become negative

In [4]:
variable = 'v'                                      # 👉 CUSTOMIZE the variable
constant = 1/10                                     # 👉 CUSTOMIZE the constant

val_save = data[variable].copy()                    # save the current values
data[variable] *= float(constant)                   # modify the data
if variable in pos_vars:
    data[variable] = check_negative(                # check for negative values
    data[variable], val_save, variable)
else:
    print(f"✓ Forcing variable '{variable}' modified")
display(data.round(4))                              # display the data

✓ Forcing variable 'v' modified


,Datetime,u,v,Tair,sol,vap,cloud,rain
0,2023-06-01 00:00:00,-3.8205,-0.1933,16.546,1.0,12.6,0.1731,0.0
1,2023-06-01 01:00:02,-3.9271,-0.1543,15.746,1.0,12.7,0.1427,0.0
2,2023-06-01 01:59:57,-3.7162,-0.2080,15.546,1.0,13.0,0.1218,0.0
3,2023-06-01 03:00:00,-3.8450,-0.1589,14.946,1.0,13.0,0.1135,0.0
4,2023-06-01 04:00:02,-3.7118,-0.1762,14.346,3.0,13.1,0.1005,0.0
...,...,...,...,...,...,...,...,...
716,2023-06-30 19:59:57,-4.1476,0.0351,18.046,2.0,15.4,0.6593,0.0
717,2023-06-30 21:00:00,-3.7128,-0.0363,17.746,1.0,14.9,0.6577,0.0
718,2023-06-30 22:00:02,-4.3388,0.0409,17.746,1.0,14.3,0.6590,0.0
719,2023-06-30 22:59:57,-3.7690,0.0220,17.146,1.0,15.3,0.7016,0.0


### Set to constant

Here you set any chosen forcing input to a constant in time.

**👉 CUSTOMIZE (next cell)** You can change these parameters:
- `variable`: choose the variable to modify from:
    - `'u'`, `'v'`, `'Tair'`, `'sol'`, `'vap'`, `'cloud'`, `'rain'`
- `constant`: the constant value of the variable
    - any real number (can also be negative and/or a fraction)
    - variables `'sol'`, `'vap'`, `'cloud'` and `'rain'` cannot become negative

In [5]:
variable = 'Tair'                                          # 👉 CUSTOMIZE the variable to modify
constant = 20                                              # 👉 CUSTOMIZE the constant added

val_save = data[variable].copy()                           # save the current values
data[variable] = np.full((len(data)), float(constant))     # modify the data
if variable in pos_vars:
    data[variable] = check_negative(                       # check for negative values
    data[variable], val_save, variable)
else:
    print(f"✓ Forcing variable '{variable}' modified")
display(data.round(4))                                     # display the data

✓ Forcing variable 'Tair' modified


,Datetime,u,v,Tair,sol,vap,cloud,rain
0,2023-06-01 00:00:00,-3.8205,-0.1933,20.0,1.0,12.6,0.1731,0.0
1,2023-06-01 01:00:02,-3.9271,-0.1543,20.0,1.0,12.7,0.1427,0.0
2,2023-06-01 01:59:57,-3.7162,-0.2080,20.0,1.0,13.0,0.1218,0.0
3,2023-06-01 03:00:00,-3.8450,-0.1589,20.0,1.0,13.0,0.1135,0.0
4,2023-06-01 04:00:02,-3.7118,-0.1762,20.0,3.0,13.1,0.1005,0.0
...,...,...,...,...,...,...,...,...
716,2023-06-30 19:59:57,-4.1476,0.0351,20.0,2.0,15.4,0.6593,0.0
717,2023-06-30 21:00:00,-3.7128,-0.0363,20.0,1.0,14.9,0.6577,0.0
718,2023-06-30 22:00:02,-4.3388,0.0409,20.0,1.0,14.3,0.6590,0.0
719,2023-06-30 22:59:57,-3.7690,0.0220,20.0,1.0,15.3,0.7016,0.0


### Add linear trend

Here you add a linear trend to any chosen forcing input.

**👉 CUSTOMIZE (next cell)** You can change these parameters:
- `variable`: choose the variable to modify from:
    - `'u'`, `'v'`, `'Tair'`, `'sol'`, `'vap'`, `'cloud'`, `'rain'`
- `step`: the time of change
    - any positive integer
    - ususally `1` corresponds to one hour
    - accordingly `24` corresponds to one day
    -  `8760` corresponds to one year
- `slope`: the amount of change per time of change
    - any real number (can also be negative and/or a fraction)
    - variables `'sol'`, `'vap'`, `'cloud'` and `'rain'` cannot become negative
    - `variable` increases by `slope` every `step` hours
- `offset`: the offset to the trend
    - any real number (can also be negative and/or a fraction)
    - variables `'sol'`, `'vap'`, `'cloud'` and `'rain'` cannot become negative
    - default is `0`

In [6]:
variable = 'Tair'                                          # 👉 CUSTOMIZE the variable to modify
step = 24                                                  # 👉 CUSTOMIZE the time of change
slope = 1/30                                               # 👉 CUSTOMIZE the slope of the trend
offset = 0                                                 # 👉 CUSTOMIZE the offset of the trend

rate = float(slope / step)                                 # the rate of change
val_save = data[variable].copy()                           # save the current values
data[variable] = (data[variable] +                         # modify the data
    np.linspace(0, rate * (len(data) - 1), len(data)) +    # add the linear increase
    float(offset))                                         # add the offset
if variable in pos_vars:
    data[variable] = check_negative(                       # check for negative values
    data[variable], val_save, variable)
else:
    print(f"✓ Forcing variable '{variable}' modified")
display(data.round(4))                                     # display the data

✓ Forcing variable 'Tair' modified


,Datetime,u,v,Tair,sol,vap,cloud,rain
0,2023-06-01 00:00:00,-3.8205,-0.1933,20.0000,1.0,12.6,0.1731,0.0
1,2023-06-01 01:00:02,-3.9271,-0.1543,20.0014,1.0,12.7,0.1427,0.0
2,2023-06-01 01:59:57,-3.7162,-0.2080,20.0028,1.0,13.0,0.1218,0.0
3,2023-06-01 03:00:00,-3.8450,-0.1589,20.0042,1.0,13.0,0.1135,0.0
4,2023-06-01 04:00:02,-3.7118,-0.1762,20.0056,3.0,13.1,0.1005,0.0
...,...,...,...,...,...,...,...,...
716,2023-06-30 19:59:57,-4.1476,0.0351,20.9944,2.0,15.4,0.6593,0.0
717,2023-06-30 21:00:00,-3.7128,-0.0363,20.9958,1.0,14.9,0.6577,0.0
718,2023-06-30 22:00:02,-4.3388,0.0409,20.9972,1.0,14.3,0.6590,0.0
719,2023-06-30 22:59:57,-3.7690,0.0220,20.9986,1.0,15.3,0.7016,0.0


# Step 4
## Save modified data

Here you save the modified forcing data to a new or the existing file.

**👉 CUSTOMIZE (next cell)** You can change this parameter:
- `copy`: Wheter to create a new forcing file (with _modified appended to the file_name) or to overwrite the original
    - `True` to create a new file (default)
    - `False` to overwrite the original

In [7]:
save_forcing_data(
    data,                        # save the modified data
    path = local_path,           # input path to the local file
    file = file_name,            # input name of the local file
    copy = True                  # 👉 CUSTOMIZE whether you want to copy and modify or overwrite the original
)

Writing modified data into: ../geneva\Forcing_modified.dat
✓ Modified 721 rows
